In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_all = sps.hstack([ICM1, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid

In [23]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender

In [14]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender_1 = RP3betaRecommender(URM_train)
recommender_2 = ItemKNNCBFRecommender(URM_train, ICM_weighted)
recommender3 = IALSRecommender_Hybrid(URM_train, ICM_all)

model_init(recommender1, 'SLIM_Pure', models_to_combine_best['SLIMER'])
model_init(recommender_1, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender_2, 'KNNweigh', models_to_combine_best['icm_weighted'])
model_init(recommender3, 'IALS_Hyb', models_to_combine_best['IALS_Hyb'])

recommender2 = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_1.W_sparse, recommender_2.W_sparse)
recommender2.fit(0.9546136842105264)

ItemKNNCBFRecommender: ICM Detected 22 ( 0.1%) items with no features.
../models_temp/Similarity_Hybrid/SLIM_Pure.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIM_Pure'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/RP3beta.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/RP3beta'
RP3betaRecommender: Loading complete
../models_temp/Similarity_Hybrid/KNNweigh.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/KNNweigh'
ItemKNNCBFRecommender: Loading complete
../models_temp/Similarity_Hybrid/IALS_Hyb.zip
Model found!
IALSRecommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/IALS_Hyb'
IALSRecommender_Hybrid: Loading complete


In [15]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [30]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3])

EvaluatorHoldout: Processed 13643 (100.0%) in 37.45 sec. Users per second: 364
SLIMElasticNetRecommender 0.2495867582538408
EvaluatorHoldout: Processed 13643 (100.0%) in 1.01 min. Users per second: 226
ItemKNNSimilarityHybridRecommender 0.22369590661645253
EvaluatorHoldout: Processed 13643 (100.0%) in 7.97 sec. Users per second: 1712
IALSRecommender_Hybrid 0.2323499114029067


In [31]:
import matplotlib.pyplot as plt
import numpy as np

In [33]:
import numpy as np

def test_percentage(recommender_a, recommender_b, recommender_c, evaluator, high1, low1, high2, low2, high3, low3, step):
    recommender = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_a, recommender_b, recommender_c)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    beta_space = np.linspace(high2, low2, step, True)
    gamma_space = np.linspace(high3, low3, step, True)
    for alp in alp_space:
        for bet in beta_space:
            for gamma in gamma_space:
                recommender.fit(alp, bet, gamma)
                r_d, _ = evaluator.evaluateRecommender(recommender)
                print(alp, bet, gamma, ":", r_d.loc[10]['MAP'])
                results.append(r_d.loc[10]['MAP'])
    
    return alp_space, beta_space, gamma_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, recommender3, evaluator_validation, 0.5, 1, 0.1, 0.3, 0.01, 0.2, 25)

_ = plt.figure(figsize=(16, 9))
plt.plot(results, alp_space, label='alpha')
plt.plot(results, beta_space, label='beta')
plt.plot(results, gamma_space, label='gamma')
plt.ylabel('MAP')
plt.xlabel('hyper')
plt.legend()
plt.show()

CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.819672131147541
ItemKNNSimilarityHybridRecommender with weight beta: 0.1639344262295082
IALSRecommender_Hybrid with weight gamma: 0.01639344262295082
EvaluatorHoldout: Processed 13643 (100.0%) in 1.57 min. Users per second: 145
0.5 0.1 0.01 : 0.25007509519970156
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.8091706001348618
ItemKNNSimilarityHybridRecommender with weight beta: 0.16183412002697237
IALSRecommender_Hybrid with weight gamma: 0.028995279838165883
EvaluatorHoldout: Processed 13643 (100.0%) in 1.61 min. Users per second: 142
0.5 0.1 0.017916666666666668 : 0.2502300615816704
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.7989347536617842
ItemKNNSimilarityHybridRecommender with weight beta: 0.15978695073235685
IALSRecommender_Hybrid with weight gamma: 0.041278295605858856
EvaluatorHoldout: Processed 13643 (100.0%) in 1.57 min. Users per second: 145
0.5 0.1 0.02583

0.5 0.1 0.19208333333333336 : 0.24877541131041106
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.625
ItemKNNSimilarityHybridRecommender with weight beta: 0.125
IALSRecommender_Hybrid with weight gamma: 0.25
EvaluatorHoldout: Processed 13643 (100.0%) in 1.57 min. Users per second: 145
0.5 0.1 0.2 : 0.2485514811363234
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.8086253369272236
ItemKNNSimilarityHybridRecommender with weight beta: 0.17520215633423178
IALSRecommender_Hybrid with weight gamma: 0.016172506738544472
EvaluatorHoldout: Processed 13643 (100.0%) in 1.54 min. Users per second: 147
0.5 0.10833333333333334 0.01 : 0.2501491781935973
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.7984031936127743
ItemKNNSimilarityHybridRecommender with weight beta: 0.17298735861610112
IALSRecommender_Hybrid with weight gamma: 0.028609447771124414
EvaluatorHoldout: Processed 13643 (100.0%) in 1.56 min. Users per second: 145
0.5 0.1

0.5 0.10833333333333334 0.17625000000000002 : 0.24903851501264118
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.6309148264984227
ItemKNNSimilarityHybridRecommender with weight beta: 0.13669821240799157
IALSRecommender_Hybrid with weight gamma: 0.23238696109358573
EvaluatorHoldout: Processed 13643 (100.0%) in 1.54 min. Users per second: 147
0.5 0.10833333333333334 0.1841666666666667 : 0.24884299640841498
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.62467464862051
ItemKNNSimilarityHybridRecommender with weight beta: 0.1353461738677772
IALSRecommender_Hybrid with weight gamma: 0.23997917751171263
EvaluatorHoldout: Processed 13643 (100.0%) in 1.58 min. Users per second: 144
0.5 0.10833333333333334 0.19208333333333336 : 0.2486859910716471
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.6185567010309279
ItemKNNSimilarityHybridRecommender with weight beta: 0.13402061855670103
IALSRecommender_Hybrid with weight gamma: 0.247

EvaluatorHoldout: Processed 13643 (100.0%) in 1.59 min. Users per second: 143
0.5 0.11666666666666667 0.16041666666666668 : 0.24918857743199907
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.6369426751592356
ItemKNNSimilarityHybridRecommender with weight beta: 0.14861995753715498
IALSRecommender_Hybrid with weight gamma: 0.21443736730360938
EvaluatorHoldout: Processed 13643 (100.0%) in 1.45 min. Users per second: 157
0.5 0.11666666666666667 0.16833333333333336 : 0.24918986886699288
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.630583289542827
ItemKNNSimilarityHybridRecommender with weight beta: 0.14713610089332632
IALSRecommender_Hybrid with weight gamma: 0.22228060956384657
EvaluatorHoldout: Processed 13643 (100.0%) in 1.52 min. Users per second: 150
0.5 0.11666666666666667 0.17625000000000002 : 0.24899512105167043
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.6243496357960457
ItemKNNSimilarityHybridRecommender wit

EvaluatorHoldout: Processed 13643 (100.0%) in 1.57 min. Users per second: 145
0.5 0.125 0.15250000000000002 : 0.2493095156653392
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.636604774535809
ItemKNNSimilarityHybridRecommender with weight beta: 0.15915119363395225
IALSRecommender_Hybrid with weight gamma: 0.20424403183023876
EvaluatorHoldout: Processed 13643 (100.0%) in 1.58 min. Users per second: 144
0.5 0.125 0.16041666666666668 : 0.24918669845225605
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.6302521008403361
ItemKNNSimilarityHybridRecommender with weight beta: 0.15756302521008403
IALSRecommender_Hybrid with weight gamma: 0.21218487394957986
EvaluatorHoldout: Processed 13643 (100.0%) in 1.57 min. Users per second: 145
0.5 0.125 0.16833333333333336 : 0.2491121907973044
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.62402496099844
ItemKNNSimilarityHybridRecommender with weight beta: 0.15600624024961
IALSRecommende

...

EvaluatorHoldout: Processed 13643 (100.0%) in 1.59 min. Users per second: 143
0.5 0.225 0.16833333333333336 : 0.24898088618036598
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5547850208044383
ItemKNNSimilarityHybridRecommender with weight beta: 0.24965325936199723
IALSRecommender_Hybrid with weight gamma: 0.19556171983356452
EvaluatorHoldout: Processed 13643 (100.0%) in 1.47 min. Users per second: 154
0.5 0.225 0.17625000000000002 : 0.24883682718854588
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5499541704857929
ItemKNNSimilarityHybridRecommender with weight beta: 0.24747937671860679
IALSRecommender_Hybrid with weight gamma: 0.2025664527956004
EvaluatorHoldout: Processed 13643 (100.0%) in 1.60 min. Users per second: 142
0.5 0.225 0.1841666666666667 : 0.24870168317027536
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5452067242162654
ItemKNNSimilarityHybridRecommender with weight beta: 0.2453430258973194
IALSRecom

EvaluatorHoldout: Processed 13643 (100.0%) in 1.63 min. Users per second: 139
0.5 0.23333333333333334 0.15250000000000002 : 0.2493290122616529
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5594405594405594
ItemKNNSimilarityHybridRecommender with weight beta: 0.26107226107226106
IALSRecommender_Hybrid with weight gamma: 0.1794871794871795
EvaluatorHoldout: Processed 13643 (100.0%) in 1.57 min. Users per second: 145
0.5 0.23333333333333334 0.16041666666666668 : 0.2490926796578055
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.55452865064695
ItemKNNSimilarityHybridRecommender with weight beta: 0.2587800369685767
IALSRecommender_Hybrid with weight gamma: 0.18669131238447323
EvaluatorHoldout: Processed 13643 (100.0%) in 1.60 min. Users per second: 142
0.5 0.23333333333333334 0.16833333333333336 : 0.24896511845716582
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5497022446174988
ItemKNNSimilarityHybridRecommender with wei

EvaluatorHoldout: Processed 13643 (100.0%) in 1.52 min. Users per second: 150
0.5 0.24166666666666667 0.1366666666666667 : 0.2495716478826869
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5641748942172073
ItemKNNSimilarityHybridRecommender with weight beta: 0.27268453220498357
IALSRecommender_Hybrid with weight gamma: 0.16314057357780914
EvaluatorHoldout: Processed 13643 (100.0%) in 1.47 min. Users per second: 155
0.5 0.24166666666666667 0.14458333333333334 : 0.24941229818419677
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.559179869524697
ItemKNNSimilarityHybridRecommender with weight beta: 0.27027027027027023
IALSRecommender_Hybrid with weight gamma: 0.17054986020503263
EvaluatorHoldout: Processed 13643 (100.0%) in 1.54 min. Users per second: 148
0.5 0.24166666666666667 0.15250000000000002 : 0.24932474819926398
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5542725173210161
ItemKNNSimilarityHybridRecommender with 

EvaluatorHoldout: Processed 13643 (100.0%) in 1.63 min. Users per second: 140
0.5 0.25 0.12875 : 0.24960668823712107
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5639097744360902
ItemKNNSimilarityHybridRecommender with weight beta: 0.2819548872180451
IALSRecommender_Hybrid with weight gamma: 0.15413533834586468
EvaluatorHoldout: Processed 13643 (100.0%) in 1.66 min. Users per second: 137
0.5 0.25 0.1366666666666667 : 0.24950665147194528
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5589194224499302
ItemKNNSimilarityHybridRecommender with weight beta: 0.2794597112249651
IALSRecommender_Hybrid with weight gamma: 0.1616208663251048
EvaluatorHoldout: Processed 13643 (100.0%) in 1.61 min. Users per second: 142
0.5 0.25 0.14458333333333334 : 0.24942826078610048
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5540166204986149
ItemKNNSimilarityHybridRecommender with weight beta: 0.27700831024930744
IALSRecommender_Hybrid wi

0.5 0.2583333333333333 0.11291666666666667 : 0.24970837419968892
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5687203791469194
ItemKNNSimilarityHybridRecommender with weight beta: 0.2938388625592417
IALSRecommender_Hybrid with weight gamma: 0.13744075829383887
EvaluatorHoldout: Processed 13643 (100.0%) in 1.88 min. Users per second: 121
0.5 0.2583333333333333 0.12083333333333333 : 0.24963840111040098
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5636449037106623
ItemKNNSimilarityHybridRecommender with weight beta: 0.29121653358384214
IALSRecommender_Hybrid with weight gamma: 0.14513856270549555
EvaluatorHoldout: Processed 13643 (100.0%) in 1.24 min. Users per second: 183
0.5 0.2583333333333333 0.12875 : 0.24954480406836926
CURRENT CONFIGURATION:
SLIMElasticNetRecommender with weight alpha: 0.5586592178770949
ItemKNNSimilarityHybridRecommender with weight beta: 0.2886405959031657
IALSRecommender_Hybrid with weight gamma: 0.152700186219739